# NFNet Inference with AMD MIGraphX


Normalizer-Free ResNet is a new residual convolutional network providing new state-of-the-art Top-1 accuracy of 86.5% at ImageNet dataset. The most important feature of the model is removing batch normalization. Instead of batch normalization, it uses adaptive gradient clipping, to provide same regularization effect of BatchNorm. <br> Details of this network: https://arxiv.org/abs/2102.06171

In this notebook, we are showing: <br>
- How to convert NFNet PyTorch model to ONNX.
- How to optimize NFNet ONNX model with AMD MIGraphX.
- How to run inference on AMD GPU with the optimized ONNX model.

The NFNet utilized in this example is the smallest NFNet version, F0: 71.5M parameters (83.6% top-1 accuracy on ImageNet)

## Requirements
Install python3-opencv package first as it provides libraries required for cv2 package.<br>It is recommended you install it through terminal, as it asks for additional inputs.

`apt-get update & apt-get install python3-opencv`

In [1]:
!pip3 install --upgrade pip #needed for opencv-python installation
!pip3 install -r requirements_nfnet.txt

In [29]:
import numpy as np
import cv2
import json
from PIL import Image
import time
from os import path 

### Importing AMD MIGraphX Python Module

In [3]:
import migraphx

### Create NFNet ONNX file
Following repository provides functionality to create NFNet ONNX file from PyTorch model.

In [4]:

# TODO: clean-up this flow.
!git clone https://github.com/cagery/pytorch-image-models.git

fatal: destination path 'pytorch-image-models' already exists and is not an empty directory.


In [5]:
!pip3 install -r ./pytorch-image-models/requirements.txt
!python3 ./pytorch-image-models/pt_to_onnx.py

Traceback (most recent call last):
  File "./pytorch-image-models/pt_to_onnx.py", line 51, in <module>
    testOnnxFile()
  File "./pytorch-image-models/pt_to_onnx.py", line 47, in testOnnxFile
    onnx_model = onnx.load("../dm_nfnet_f0.onnx")
  File "/usr/local/lib/python3.6/dist-packages/onnx/__init__.py", line 114, in load_model
    s = _load_bytes(f)
  File "/usr/local/lib/python3.6/dist-packages/onnx/__init__.py", line 30, in _load_bytes
    with open(cast(Text, f), 'rb') as readable:
FileNotFoundError: [Errno 2] No such file or directory: '../dm_nfnet_f0.onnx'


### Load ImageNet labels

In [10]:
with open('../python_api_inference/imagenet_simple_labels.json') as json_data:
    labels = json.load(json_data)

In [11]:
## Load ONNX model using MIGraphX

In [12]:
model = migraphx.parse_onnx("dm_nfnet_f0.onnx")
model.compile(migraphx.get_target("gpu"))

print(model.get_parameter_names())
print(model.get_parameter_shapes())
print(model.get_output_shapes())

### Functions for image processing

In [36]:
def make_nxn(image, n):
    height, width = image.shape[:2]    
    if height > width:
        dif = height - width
        bar = dif // 2 
        square = image[(bar + (dif % 2)):(height - bar),:]
        return cv2.resize(square, (n, n))
    elif width > height:
        dif = width - height
        bar = dif // 2
        square = image[:,(bar + (dif % 2)):(width - bar)]
        return cv2.resize(square, (n, n))
    else:
        return cv2.resize(image, (n, n))
    
def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):  
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

def input_process(frame, dim):
    # Crop and resize original image
    cropped = make_nxn(frame, dim)
    # Convert from HWC to CHW
    chw = cropped.transpose(2,0,1)
    # Apply normalization
    pp = preprocess(chw)
    # Add singleton dimension (CHW to NCHW)
    data = np.expand_dims(pp.astype('float32'),0)
    return data

### Download example image

In [14]:
# Fetch example image
!wget http://farm5.static.flickr.com/4072/4462811418_8bc2bd42ca_z_d.jpg -O traffic_light.jpg
# Read the image
im = cv2.imread('traffic_light.jpg')  

--2021-04-22 22:26:52--  http://farm5.static.flickr.com/4072/4462811418_8bc2bd42ca_z_d.jpg
Resolving farm5.static.flickr.com (farm5.static.flickr.com)... 143.204.162.96, 2600:9000:20ef:2000:0:5a51:64c9:c681, 2600:9000:20ef:2200:0:5a51:64c9:c681, ...
Connecting to farm5.static.flickr.com (farm5.static.flickr.com)|143.204.162.96|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://farm5.static.flickr.com/4072/4462811418_8bc2bd42ca_z_d.jpg [following]
--2021-04-22 22:26:52--  https://farm5.static.flickr.com/4072/4462811418_8bc2bd42ca_z_d.jpg
Connecting to farm5.static.flickr.com (farm5.static.flickr.com)|143.204.162.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘traffic_light.jpg’

traffic_light.jpg       [ <=>                ]  52.04K  --.-KB/s    in 0.02s   

2021-04-22 22:26:52 (2.23 MB/s) - ‘traffic_light.jpg’ saved [53289]



In [26]:
# Process the read image to conform input requirements
data_input = input_process(im, 192)

# Run the model
start = time.time()
results = model.run({'inputs':data_input}) # Your first inference would take longer than the following ones.
print(f"Time inference took: {100*(time.time() - start):.2f}ms")
# Extract the index of the top prediction
res_npa = np.array(results[0])
print(f"\nResult: {labels[np.argmax(res_npa)]}")

Time inference took: 2.75ms

Result: traffic light
